In [1]:
import os
import django

# The following lines are here to avoid `SynchronousOnlyOperation` exception
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from articles.models import Article

In [2]:
from spacy.lang.ky import Kyrgyz

spacy_nlp = Kyrgyz()
spacy_nlp.add_pipe("sentencizer")

In [3]:
arts = Article.objects.all()
print(len(arts))

30934


In [4]:
# Clean the file
f = open('all_texts.txt', 'w')
f.write('')
f.close()

In [5]:
def append_article_to_file(article, sentences):
    with open('all_texts.txt', 'a') as file:
        # Formatting the article data
        title = article.title.strip() if article.title else 'No Title'
        article_url = article.article_url.strip() if article.article_url else 'No URL'
        created_at = article.created_at.strftime('%Y-%m-%d %H:%M:%S') if article.created_at else 'No Date'
        posted_by = article.posted_by.strip() if article.posted_by else 'Unknown'
        author = article.author.strip() if article.author else 'Unknown'
        translator = article.translator.strip() if article.translator else 'Unknown'
        editor = article.editor.strip() if article.editor else 'Unknown'

        # Writing the article metadata to the file
        file.write(f'Title: {title}\n')
        file.write(f'URL: {article_url}\n')
        file.write(f'Created At: {created_at}\n')
        file.write(f'Posted By: {posted_by}\n')
        file.write(f'Author: {author}\n')
        file.write(f'Translator: {translator}\n')
        file.write(f'Editor: {editor}\n')
        file.write('Sentences:\n')
        file.write('-' * 40 + '\n')

        # Writing the sentences to the file
        for sentence in sentences:
            text = sentence.text.strip()
            file.write(text + '\n')

        file.write('-' * 40 + '\n')  # Separator for readability

In [6]:
# Append each article to all_texts.txt
for art in arts:
    doc = spacy_nlp(art.text)
    append_article_to_file(art, doc.sents)

In [7]:
import zipfile

def compress_file(filename):
    with zipfile.ZipFile(filename + '.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zip_file:
        zip_file.write(filename)


In [8]:
# Put the file into an archive
compress_file('all_texts.txt')

In [20]:
# Remove all_texts.txt since github won't accept big files
!rm all_texts.txt

In [21]:
# Decompress the archive. Run this if only you want to work with the raw text file

import zipfile

with zipfile.ZipFile('all_texts.txt.zip', 'r') as zip_ref:
    zip_ref.extract('all_texts.txt')
